# Kaggle Student Performance

This notebook contains EDA and some preliminary model experimentation.

In [12]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
file = "/content/drive/My Drive/Colab Notebooks/StudentsPerformance.csv"

Mounted at /content/drive


In [13]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [14]:
data = pd.read_csv(file)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [ ]:
list(zip(data.columns, data.dtypes, data.nunique()))

[('gender', dtype('O'), 2),
 ('race/ethnicity', dtype('O'), 5),
 ('parental level of education', dtype('O'), 6),
 ('lunch', dtype('O'), 2),
 ('test preparation course', dtype('O'), 2),
 ('math score', dtype('int64'), 81),
 ('reading score', dtype('int64'), 72),
 ('writing score', dtype('int64'), 77)]

In [ ]:
data.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


With additional time, I would prioritize more EDA.

### Data Prep

In [16]:
# Rename long column names
data.rename(columns={"race/ethnicity": "ethnicity",
                     "parental level of education": "parent_education",
                     "math score": "math_score",
                     "reading score": "reading_score",
                     "writing score": "writing_score",
                     "test preparation course": "test_prep"},
            inplace=True)

In [17]:
data.columns

Index(['gender', 'ethnicity', 'parent_education', 'lunch', 'test_prep',
       'math_score', 'reading_score', 'writing_score'],
      dtype='object')

In [20]:
# String column values to numeric
# We aren't doing any scaling or other transformations, so there isn't a risk of data leakage doing this all at once
for c in data.columns[:-3]:
  data[c] = le.fit_transform(data[c])

In [21]:
data.head()

,gender,ethnicity,parent_education,lunch,test_prep,math_score,reading_score,writing_score
0,0,1,1,1,1,72,72,74
1,0,2,4,1,0,69,90,88
2,0,1,3,1,1,90,95,93
3,1,0,0,0,1,47,57,44
4,1,2,4,1,1,76,78,75


In [ ]:
data.describe()

,gender,ethnicity,parent_education,lunch,test_prep,math_score,reading_score,writing_score
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000
mean,0.482000,2.174000,2.486000,0.645000,0.642000,66.08900,69.169000,68.054000
std,0.499926,1.157179,1.829522,0.478753,0.479652,15.16308,14.600192,15.195657
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,17.000000,10.000000
25%,0.000000,1.000000,1.000000,0.000000,0.000000,57.00000,59.000000,57.750000
50%,0.000000,2.000000,2.000000,1.000000,1.000000,66.00000,70.000000,69.000000
75%,1.000000,3.000000,4.000000,1.000000,1.000000,77.00000,79.000000,79.000000
max,1.000000,4.000000,5.000000,1.000000,1.000000,100.00000,100.000000,100.000000


In [22]:
# train-test split
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(data, test_size=0.3)

In [23]:
# I'm not sure whether to include reading_score in the features, so I'm
# going to assume we wouldn't have access to this for the purpose of the exercise
features = data.columns[:-3]
targets = ['math_score', 'writing_score']

We are building on regression model to predict two dependent variables. Let's see if they are correlated.


In [ ]:
data['writing_score'].corr(data['math_score'])

0.8026420459498085

That looks pretty correlated to me, so I'm going to go for multivariate regression vs. independent models.

### Initial model tests

**Note: Normally I would do these kinds of experiments using MLFlow or another experiment tracker. However, I don't have that set up on my home machine at the moment, so I'm going to do this the old fashioned way.**

First, I'm going to try a few sklearn options that directly support multioutput regression:
- LinearRegression
- [PLSRegression](https://en.wikipedia.org/wiki/Partial_least_squares_regression)
- KNeighborsRegressor
- RandomForestRegressor

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import RegressorChain

In [28]:
def mae_with_cv(model):
  """Evaluates model performance (MAE) using k-fold cross validation.

  Args:
    model: An sklearn model
  """
  cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
  n_scores = cross_val_score(model, train_df[features], train_df[targets], scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
  n_scores = np.absolute(n_scores)
  print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
print("MAE (sd) for Linear Regression")
mae_with_cv(LinearRegression())

MAE (sd) for Linear Regression
MAE: 10.421 (0.872)


In [ ]:
print("MAE (sd) for PLS Regression")
mae_with_cv(PLSRegression())

MAE (sd) for PLS Regression
MAE: 10.425 (0.861)


In [ ]:
print("MAE (sd) for KNN Regression")
mae_with_cv(KNeighborsRegressor())

MAE (sd) for KNN Regression
MAE: 11.253 (0.815)


In [ ]:
print("MAE (sd) for Random Forest Regression")
mae_with_cv(RandomForestRegressor())

MAE (sd) for Random Forest Regression
MAE: 11.443 (0.804)


Next, I'm going to try a model that don't natively support multiple outputs: SVM. We can do this in two ways:
- Naively, using two separate models each with one target. This assumes the outputs are independent of each other, which is not the case in this situation.
- Using a sequence of linear models, where the first model in the sequence uses the input and predicts one output, and the second model uses the input and the output from the first model to make a prediction. I'm going with this approach below.

In [ ]:
print("MAE (sd) for SVR")
mae_with_cv(RegressorChain(LinearSVR()))

MAE (sd) for SVR
MAE: 11.085 (0.901)


In general, these models are performing pretty similarly. Some models have more to gain from hyperparam tuning than others, so since the data set is small it probably makes sense to try tuning them all, and then comparing again.

### Hyperparam tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Linear Regression
lr_cv = RandomizedSearchCV(
    estimator=LinearRegression(),
    param_distributions={
              'copy_X': [True, False],
              'fit_intercept': [True, False],
              'positive': [True, False],
          },
    n_iter=8,
    cv=3
)

lr_cv.fit(train_df[features], train_df[targets])
lr_cv.best_params_

In [ ]:
print("MAE (sd) for tuned Linear Regression")
mae_with_cv(lr_cv)

MAE (sd) for tuned Linear Regression
MAE: 10.421 (0.872)


In [ ]:
# PLS Regression
plsr_cv = RandomizedSearchCV(
    estimator=PLSRegression(),
    param_distributions={
              'scale': [True, False],
              'n_components': [1, 2]
          },
    n_iter=4,
    cv=3
)
plsr_cv.fit(train_df[features], train_df[targets])
plsr_cv.best_params_

{'n_components': 2, 'scale': True}

In [ ]:
print("MAE (sd) for tuned PLS Regression")
mae_with_cv(plsr_cv)

MAE (sd) for tuned PLS Regression
MAE: 10.425 (0.861)


In [ ]:
# KNN Regression
knn_cv = RandomizedSearchCV(
    estimator=KNeighborsRegressor(),
    param_distributions={
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'leaf_size': [int(x) for x in np.linspace(start=5, stop=50, num=10)],
              'n_neighbors':  [int(x) for x in np.linspace(start=1, stop=20, num=20)],
              'weights': ['uniform', 'distance']
          },
    n_iter=100,
    cv=3
)

knn_cv.fit(train_df[features], train_df[targets])
knn_cv.best_params_

{'algorithm': 'brute',
 'leaf_size': 10,
 'n_neighbors': 16,
 'weights': 'uniform'}

In [ ]:
print("MAE (sd) for tuned KNN Regression")
mae_with_cv(knn_cv)

MAE (sd) for tuned KNN Regression
MAE: 10.932 (0.841)


In [ ]:
# Random Forest Regression
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)

rf_cv = RandomizedSearchCV(
    estimator=RandomForestRegressor(),
    param_distributions={
               'n_estimators': [int(x) for x in np.linspace(start=200, stop=2000, num=10)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': max_depth,
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]
          },
    n_iter=100,
    cv=3
)

rf_cv.fit(train_df[features], train_df[targets])
rf_cv.best_params_

{'bootstrap': True,
 'max_depth': 40,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 400}

In [ ]:
rf = RandomForestRegressor()

In [ ]:
rf.set_params(**rf_cv.best_params_)

RandomForestRegressor(max_depth=40, max_features='sqrt', min_samples_leaf=4,
                      min_samples_split=10, n_estimators=400)

In [ ]:
print("MAE (sd) for tuned Random Forest Regression")
mae_with_cv(rf)

MAE (sd) for tuned Random Forest Regression
MAE: 10.604 (0.925)


In [ ]:
# SVM
svm_cv = RandomizedSearchCV(
    estimator=RegressorChain(LinearSVR()),
    param_distributions={
              'base_estimator__C': [float(x) for x in np.linspace(start=0, stop=1, num=20)],
              'base_estimator__dual': [True, False],
              'base_estimator__epsilon':  [float(x) for x in np.linspace(start=0, stop=1, num=20)],
              'base_estimator__fit_intercept': [True, False],
              'base_estimator__loss': ['epsilon_insensitive', 'squared_epsilon_insensitive'],

          },
    n_iter=1000,
    cv=3
)

svm_cv.fit(train_df[features], train_df[targets])

In [25]:
svm_cv.best_params_

{'base_estimator__C': 0.9473684210526315,
 'base_estimator__dual': False,
 'base_estimator__epsilon': 0.5789473684210527,
 'base_estimator__fit_intercept': True,
 'base_estimator__loss': 'squared_epsilon_insensitive'}

In [26]:
svm = RegressorChain(LinearSVR())
svm.set_params(**svm_cv.best_params_)

RegressorChain(base_estimator=LinearSVR(C=0.9473684210526315, dual=False,
                                        epsilon=0.5789473684210527,
                                        loss='squared_epsilon_insensitive'))

In [ ]:
print("MAE (sd) for tuned SVM")
mae_with_cv(svm)

MAE (sd) for tuned SVM
MAE: 10.411 (0.841)


Ok, SVM is our winner!